## 8 - Smart Pointers: `shared_ptr`
##### **Author: Adam Gatt**

The second type of smart pointer is `shared_ptr` which allows for multiple "owning" references, and the dynamic data will be deleted when the last owning reference falls out of scope. This is achieved by a reference-counting approach (much like garbage collection in the JVM). It is useful when:

* It cannot be made clear who the single authoritative owner should be
* There is a need for multiple references but no guarantees on which of them will be finalised last (e.g. a multi-threaded environment)

The behaviour of `shared_ptr` is achieved by a reference-counting approach, much like garbage collection in the JVM. This comes at a cost compared to its unique counterpart:

* The reference counter requires extra memory and prevents the smart pointer from being fully optimised away
* It is possible to create "reference cycles" of two shared_ptrs with (perhaps indirect) references to each other, preventing either of them from being cleaned up

As such, `shared_ptr` should not be used as a default approach where a `unique_ptr` would do instead. 

In [1]:
#include <memory>
#include <iostream>

class DynamicData {
    public:
    DynamicData() {
        std::cout << "Data created" << std::endl;
    }
    
    ~DynamicData() {
        std::cout << "Data cleaned up" << std::endl;
    }
}

In [2]:
{
    std::cout << "> outer scope" << std::endl;
    
    // First shared pointer, but contains no data yet
    std::shared_ptr<DynamicData> ptr1;
    
    {
        std::cout << "> inner scope" << std::endl;
        
        // Shared pointer, creating dynamic data
        std::shared_ptr<DynamicData> ptr2 = std::make_shared<DynamicData>();
    
        // Share ptr2 reference with ptr1 (via copy assignment operator)
        ptr1 = ptr2;
        std::cout << "< inner scope" << std::endl;
    }
    
    std::cout << "< outer scope" << std::endl;
}

> outer scope
> inner scope
Data created
< inner scope
< outer scope
Data cleaned up


We can call `use_count()` to determine how many shared_ptrs are currently pointing to the same data.

In [4]:
{
    std::shared_ptr<DynamicData> ptr1 = std::make_shared<DynamicData>();
    std::shared_ptr<DynamicData> ptr2 = ptr1;
    std::shared_ptr<DynamicData> ptr3 = ptr1;
    std::shared_ptr<DynamicData> ptr4 = ptr1;

    std::cout << ptr1.use_count() << std::endl;
}

Data created
4
Data cleaned up


### Optional: Smart pointer creation, `new` vs `make_*`
In the examples above I often used the `std::make_unique` function to create unique_ptrs. If I wanted to, there is no reason why I can't create a unique_ptr with [its standard constructor](https://en.cppreference.com/w/cpp/memory/unique_ptr/unique_ptr), passing it a pointer to the dynamic data that I create manually with the usual `new` keyword.

In [ ]:
#include <memory>
#include <string>

std::unique_ptr<std::string> dynamicStringManager(new std::string("Hello, World!"));

*dynamicStringManager

However, in C++14 the functions `make_unique` and `make_shared` were introduced as it was realised that there was a subtle bug that may result in a memory leak. In short, if a smart pointer was created as a temporary object (e.g. such as in a function call) and then an exception occurs before the temporary object is used and resolved, there is a chance that the dynamic data would be successfully created but the creation of the owning smart pointer would fail. With no smart pointer to clean it up, the dynamic data would leak.

In [ ]:
func1(std::unique_ptr<A>(new A("Hello", 50)), func2())

Consider the (contrived) example above, with a function "func1" that requires a smart pointer and the results of a second function "func2". In the C++ standard, the execution order of the statement is not strongly defined. It is very possible for the dynamic object `new A()` to be created first, and then for `func2` to execute and cause an exception. The temporary unique_ptr never gets created, and there is nothing responsible for deleting our dynamic object.

In the alternative syntax below, the `make_unique()` call is atomic. The unique_ptr may already have been created when func2 throws an exception, but it is firmly an existing _temporary object_ by then. When the exception occurs the stack will be unwound and the smart pointer's destructor will be called, cleaning up the data. 

In [ ]:
func1(std::make_unique<A>("Hello", 50), func2())

But then when C++17 was released, the standard was changed to include additional guarantees, including that all side-effects of a function argument must be evaluated before moving on to the next argument. This rule fixes the above example and obviates the safety-based argument for using make_unique. However, if you cannot guarantee that your code will only be compiled with C++17, it may be good to make a habit of using make_unique (and make_shared) to avoid being caught out.

In addition to the safety argument, there are two other arguments commonly made for using the make_* creation functions:
* You only need to write the underlying type name once, improving readability when the type name is especially long
* You can arguably avoid the need to write `new` completely, making the new/delete semantics effectively obsolete.



### Shared_ptr
Reference counted
#### Creation (Manual/make_shared)
#### Create extra reference
#### Drop reference
#### weak_ptr
Non-owning reference - check for null and expect to lose at any time
    Breaks cycles of shared_ptrs
Example of object cache


### Rules for smart pointers
* create smart pointers with their std::make_* functions
* return smart pointers by value
* Accept unique_ptr as parameter if you intend to claim ownership
* Accept shared_ptr as parameter if you intend to add to ownership
* Accept raw pointer if you intend to use but not claim ownership